In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from src import object_test

## tor testing
If i treat tor requests like normal requests, what happens?

In [3]:
import torrequest

search_page = torrequest.requests.get('https://www.tapology.com/search?term=ufc&mainSearchFilter=events').content

events = object_test.events()
events.create_events_df(search_page)
events.table

,bouts,date,enclosure,event,index,link,location,name,start_time,venue
0,12,2020-06-20,NaN,UFC Fight Night,22,/fightcenter/events/67608-ufc-on-espn,NaN,Blaydes vs. Volkov,NaN,NaN
1,10,2020-06-13,NaN,UFC Fight Night,23,/fightcenter/events/67562-ufc-on-espn,NaN,Eye vs. Calvillo,NaN,NaN
2,12,2020-06-06,NaN,UFC 250,24,/fightcenter/events/67364-ufc-251,NaN,Nunes vs. Spencer,NaN,NaN
3,11,2020-05-30,NaN,UFC Fight Night,25,/fightcenter/events/69127-ufc-fight-night,NaN,Woodley vs. Burns,NaN,NaN
4,11,2020-05-16,NaN,UFC Fight Night,26,/fightcenter/events/67412-ufc-on-espn-33,NaN,Overeem vs. Harris,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
498,10,1995-04-07,NaN,UFC 5,864,/fightcenter/events/ufc-5-return-of-the-beast,NaN,Return of the Beast,NaN,NaN
499,10,1994-12-16,NaN,UFC 4,865,/fightcenter/events/ufc-4-revenge-of-the-warriors,NaN,Revenge of the Warriors,NaN,NaN
500,6,1994-09-09,NaN,UFC 3,866,/fightcenter/events/ufc-3-the-american-dream,NaN,The American Dream,NaN,NaN
501,15,1994-03-11,NaN,UFC 2,867,/fightcenter/events/ufc-2-no-way-out,NaN,No Way Out,NaN,NaN


But how do I know it's anonymous? I didn't pass it my tor password or anything, so how do I know it's secure?

In [4]:
response= requests.get('http://ipecho.net/plain')
print ("My Original IP Address:",response.text)

My Original IP Address: 73.72.178.29


In [5]:
response= torrequest.requests.get('http://ipecho.net/plain')
print ("New Ip Address",response.text)

New Ip Address 73.72.178.29


### Actual import

This is how thay really want me to import it.

In [6]:
from torrequest import TorRequest


In [8]:
%%time
tr=TorRequest(password='8453')
tr.reset_identity_async() #Reset Tor
response= tr.get('http://ipecho.net/plain')
print ("New Ip Address",response.text)

New Ip Address 79.172.193.32
CPU times: user 38.1 ms, sys: 11.9 ms, total: 50 ms
Wall time: 4.34 s


In [7]:
%%time
tr=TorRequest(password='8453')
tr.reset_identity() #Reset Tor
response= tr.get('http://ipecho.net/plain')
print ("New Ip Address",response.text)

New Ip Address 199.249.230.104
CPU times: user 74.1 ms, sys: 17.3 ms, total: 91.4 ms
Wall time: 10.9 s


### Troubleshooting
I got an error like this: Process terminated: Failed to bind one of the listener ports.

In order to get this to work, I had to kill the tor process. I did this by using grep tor to find the process id numbers for both tor processes and then sudo kill [PID]

### Async tor reset test

In [9]:
import time

In [10]:
%%time
tr=TorRequest(password='8453')
tr.reset_identity_async() #Reset Tor
response= tr.get('http://ipecho.net/plain')
print ("New Ip Address",response.text)

time.sleep(15)
tr=TorRequest(password='8453')
tr.reset_identity_async() #Reset Tor
response= tr.get('http://ipecho.net/plain')
print ("New Ip Address",response.text)

New Ip Address 79.172.193.32
New Ip Address 195.254.134.194
CPU times: user 209 ms, sys: 32.8 ms, total: 242 ms
Wall time: 16.7 s


Takes about 12 seconds to reset the tor connection, so performing an asyncronous reset would most beneficial if the rest of the scraping algorithm took around that time. Otherwise, I run the rest of scraping too much of the website before the async reset catches up.

### User agent test

In [11]:
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36'
headers = {'User-Agent': user_agent} #set user agent

url = 'https://www.tapology.com/fightcenter/fighters/51080-curtis-blaydes'
response = tr.get(url, headers=headers)

fighter_soup = BeautifulSoup(response.text)

In [12]:
fighters_test = object_test.fighters()
fighters_test.create_fighters_table(fighter_soup, url)
fighters_test.table

,Name,ProMMARecord,Nickname,CurrentStreak,Age,DateofBirth,LastFight,WeightClass,LastWeigh-In,Affiliation,Height,Reach,CareerDisclosedEarnings,Born,Fightingoutof,College,FoundationStyle,FighterLinks,PersonalLinks,link
0,Curtis Blaydes,"14-2-0, 1 NC (Win-Loss-Draw)",Razor,4 Wins,29,1991.02.18,"June 20, 2020\nin\nUFC",Heavyweight,261.0 lbs,Elevation Fight Team,"6'4"" (193cm)","80.0"" (203cm)","$338,000 USD","Naperville, Illinois","Chicago, Illinois",Harper College,Wrestling,,,https://www.tapology.com/fightcenter/fighters/...


# User agent

In [18]:
from fake_useragent import UserAgent

ua = UserAgent()

ua.random

In [19]:
ua.random

'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2224.3 Safari/537.36'

In [20]:
ua.random

'Mozilla/5.0 (Windows; U; Windows NT 6.1; cs-CZ) AppleWebKit/533.20.25 (KHTML, like Gecko) Version/5.0.4 Safari/533.20.27'

In [21]:
ua.random

'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36'

In [22]:
ua.random

'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36'

In [23]:
ua.random

'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 7.0; InfoPath.3; .NET CLR 3.1.40767; Trident/6.0; en-IN)'

In [24]:
ua.random

'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1664.3 Safari/537.36'